In [2]:
import numpy as np
import wordfreq
'''
wordfreq https://pypi.org/project/wordfreq/
word_frequency(word, lang, wordlist='best', minimum=0.0)
function: tokenize()
>>> wordfreq.tokenize('l@s niñ@s', 'es')
['l@s', 'niñ@s'] #按空格分词成列表
'''

"\nwordfreq https://pypi.org/project/wordfreq/\nword_frequency(word, lang, wordlist='best', minimum=0.0)\nfunction: tokenize()\n>>> wordfreq.tokenize('l@s niñ@s', 'es')\n['l@s', 'niñ@s'] #按空格分词成列表\n"

In [3]:
vocab = {}
token_id = 1 #token_id一般从1开始，最后再加一个unknown的id,0一般对应这句话为空的情况
lengths = []

with open('./data/test.txt', 'r') as f:
    for l in f:
        tokens = wordfreq.tokenize(l.strip(), 'en')#1 text
        lengths.append(len(tokens)) # store the length of all texts
        for t in tokens:
            if t not in vocab:
                vocab[t] = token_id #construct vacab dict
                token_id += 1

In [10]:
print('共有 %d 个词'%token_id)
print('共有 %d 条text'%len(lengths))
print('vacab', vocab)


共有 54 个词
共有 4 条text
vacab {'to': 1, 'the': 2, 'world': 3, 'you': 4, 'may': 5, 'be': 6, 'one': 7, 'person': 8, 'but': 9, 'no': 10, 'man': 11, 'or': 12, 'woman': 13, 'is': 14, 'worth': 15, 'your': 16, 'tears': 17, 'and': 18, 'who': 19, 'won': 20, '鈥檛': 21, 'make': 22, 'cry': 23, 'never': 24, 'frown': 25, 'even': 26, 'when': 27, 'are': 28, 'sad': 29, 'because': 30, 'know': 31, 'falling': 32, 'in': 33, 'love': 34, 'with': 35, 'smile': 36, 'we': 37, 'met': 38, 'at': 39, 'wrong': 40, 'time': 41, 'separated': 42, 'right': 43, 'most': 44, 'urgent': 45, 'take': 46, 'beautiful': 47, 'scenery': 48, 'deepest': 49, 'wound': 50, 'was': 51, 'real': 52, 'emotions': 53}


In [11]:
x = np.zeros((len(lengths), max(lengths))) #initialize x 
l_no = 0
with open('./data/test.txt', 'r') as f:
    for l in f:
        tokens = wordfreq.tokenize(l.strip(), 'en')
        for i in range(len(tokens)):
            x[l_no, i] = vocab[tokens[i]] #construct x, 总觉得这里reread file 有点多余，也许可以优化
        l_no += 1 # text idx

In [16]:
x.shape

(4, 30)

In [14]:
x#除去最长的那个text,其他不够长的text一律补0

array([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.,  1.,  7.,  8.,  4.,
         5.,  6.,  2.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [10., 11., 12., 13., 14., 15., 16., 17., 18.,  2.,  7., 19., 14.,
        20., 21., 22.,  4., 23.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [24., 25., 26., 27.,  4., 28., 29., 30.,  4., 24., 31., 19., 14.,
        32., 33., 34., 35., 16., 36.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [37., 38., 39.,  2., 40., 41.,  9., 42., 39.,  2., 43., 41.,  2.,
        44., 45., 14.,  1., 46.,  2., 44., 47., 48.,  2., 49., 50., 51.,
         2., 44., 52., 53.]])

In [21]:
import torch
import torch.nn as nn
from torch.autograd import Variable

x=torch.Tensor(x)
x = Variable(x) #transform x to torch.autograd.Variable
'''
torch.autograd.Variable:https://pytorch-cn.readthedocs.io/zh/latest/package_references/torch-autograd/

'''
lengths = torch.Tensor(lengths) # transform lengths to torch.Tensor
_, idx_sort = torch.sort(lengths, dim=0, descending=True)
_, idx_unsort = torch.sort(idx_sort, dim=0)#idx_unsort的作用在于将batch中的序列调整为原来的顺序。
'''
torch.sort(input, dim=None, descending=False, out=None)
input:输入张量
dim:指定维度，默认为输入的最后一个维度
descending:若为True,则为降序排列
返回元组(sorted_tensor,sorted_indices),indices是index的复数形式,sorted_indices是排序后的原始输入的下标
'''

'\ntorch.sort(input, dim=None, descending=False, out=None)\ninput:输入张量\ndim:指定维度，默认为输入的最后一个维度\ndescending:若为True,则为降序排列\n返回元组(sorted_tensor,sorted_indices),indices是index的复数形式,sorted_indices是排序后的原始输入的下标\n'

In [22]:
lengths

tensor([17., 18., 19., 30.])

In [24]:
idx_sort

tensor([3, 2, 1, 0])

In [25]:
idx_unsort#??

tensor([3, 2, 1, 0])

In [26]:
#将x 和lengths 都从大到小 排序
print('x排序之前：',x)
#pack_padded_sequence
x = x.index_select(0, idx_sort)
print('x排序之后：',x)
#index_select(x, 1, indices) #索引查找
#1 代表维度1，indices筛选的索引序号
print('lengths排序之前：',lengths)
lengths = list(lengths[idx_sort])
print('lengths排序之后：',lengths)


x_packed = nn.utils.rnn.pack_padded_sequence(input=x, lengths=lengths, batch_first=True)
'''
pack_padded_sequence函数的参数:
input:x为已根据长度 从大到小 排好序
lengths:需要 从大到小 排序
batch_first:如果为true，则x的第一维为batch_size，第二维为seq_length，否则相反。
'''

x排序之前： tensor([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.,  1.,  7.,  8.,  4.,  5.,
          6.,  2.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [10., 11., 12., 13., 14., 15., 16., 17., 18.,  2.,  7., 19., 14., 20.,
         21., 22.,  4., 23.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [24., 25., 26., 27.,  4., 28., 29., 30.,  4., 24., 31., 19., 14., 32.,
         33., 34., 35., 16., 36.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [37., 38., 39.,  2., 40., 41.,  9., 42., 39.,  2., 43., 41.,  2., 44.,
         45., 14.,  1., 46.,  2., 44., 47., 48.,  2., 49., 50., 51.,  2., 44.,
         52., 53.]])
x排序之后： tensor([[37., 38., 39.,  2., 40., 41.,  9., 42., 39.,  2., 43., 41.,  2., 44.,
         45., 14.,  1., 46.,  2., 44., 47., 48.,  2., 49., 50., 51.,  2., 44.,
         52., 53.],
        [24., 25., 26., 27.,  4., 28., 29., 30.,  4., 24., 31., 19., 14., 32.,
         33., 34

'\npack_padded_sequence函数的参数:\ninput:x为已根据长度 从大到小 排好序\nlengths:需要 从大到小 排序\nbatch_first:如果为true，则x的第一维为batch_size，第二维为seq_length，否则相反。\n'

In [27]:
x_packed #我觉得应该是矩阵压缩，但我看不懂这个压缩后的矩阵

PackedSequence(data=tensor([37., 24., 10.,  1., 38., 25., 11.,  2., 39., 26., 12.,  3.,  2., 27.,
        13.,  4., 40.,  4., 14.,  5., 41., 28., 15.,  6.,  9., 29., 16.,  7.,
        42., 30., 17.,  8., 39.,  4., 18.,  9.,  2., 24.,  2.,  1., 43., 31.,
         7.,  7., 41., 19., 19.,  8.,  2., 14., 14.,  4., 44., 32., 20.,  5.,
        45., 33., 21.,  6., 14., 34., 22.,  2.,  1., 35.,  4.,  3., 46., 16.,
        23.,  2., 36., 44., 47., 48.,  2., 49., 50., 51.,  2., 44., 52., 53.]), batch_sizes=tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 2, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1]), sorted_indices=None, unsorted_indices=None)

In [28]:
#pad_packed_sequence
x_padded = nn.utils.rnn.pad_packed_sequence(x_packed, batch_first=True)
output = x_padded[0].index_select(0, idx_unsort)#把x_padded还原

In [29]:
output

tensor([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.,  1.,  7.,  8.,  4.,  5.,
          6.,  2.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [10., 11., 12., 13., 14., 15., 16., 17., 18.,  2.,  7., 19., 14., 20.,
         21., 22.,  4., 23.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [24., 25., 26., 27.,  4., 28., 29., 30.,  4., 24., 31., 19., 14., 32.,
         33., 34., 35., 16., 36.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [37., 38., 39.,  2., 40., 41.,  9., 42., 39.,  2., 43., 41.,  2., 44.,
         45., 14.,  1., 46.,  2., 44., 47., 48.,  2., 49., 50., 51.,  2., 44.,
         52., 53.]])